## Глава 1. Подготовка

Для начала нужно предобработать датасет

In [1]:
import pandas as pd
import numpy as np
import re


def is_phrase(string):
    '''Функция проверяет является ли строка фразой (есть хотя бы 1 пробел)'''
    return " " in string


def is_word(string):
    '''Функция проверяет является ли строка - русскоязычным словом'''
    pattern = re.compile("[ЁёА-я]+")
    return bool(pattern.fullmatch(string))

Надо распарсить файл такого вида:

```
  id  |        stim        |                              word                               
------+--------------------+-----------------------------------------------------------------
    1 | абсолютный         | -
    1 | абсолютный         | 0
    1 | абсолютный         | 100%
    1 | абсолютный         | 100%-ный
    1 | абсолютный         | max
```

Поэтому просто будем делить по `"|"`, и обрезать пустые пробелы.

In [2]:
data = []
with open("./data/to_tag.txt", "r") as in_f:
    for line in in_f.readlines():
        try:
            parts = line.split("|")
            a = parts[0]
            b = parts[1]
            c = "|".join(parts[2:])
            data.append(list(map(str.strip, [a, b, c])))
        except IndexError:
            pass

data = pd.DataFrame(data[1:], columns=data[0])
print(data.shape)
data.head()

(167889, 3)


,id,stim,word
0,1,абсолютный,-
1,1,абсолютный,0
2,1,абсолютный,100%
3,1,абсолютный,100%-ный
4,1,абсолютный,max


Добавим избыточную колонку `"phrase"`, которая является просто соединеным стимулом и реакцией через пробел.

И добавим колонку `auto_tag` - которая будет хранить спецобозначения фраз и НЕ русских слов, чтобы в дальнейшем их не пихать в модели

In [3]:
data["phrase"] = data.stim + " " + data.word
data["auto_tag"] = None
data

,id,stim,word,phrase,auto_tag
0,1,абсолютный,-,абсолютный -,None
1,1,абсолютный,0,абсолютный 0,None
2,1,абсолютный,100%,абсолютный 100%,None
3,1,абсолютный,100%-ный,абсолютный 100%-ный,None
4,1,абсолютный,max,абсолютный max,None
...,...,...,...,...,...
167884,1000,ярость,шар,ярость шар,None
167885,1000,ярость,эмоции,ярость эмоции,None
167886,1000,ярость,эмоция,ярость эмоция,None
167887,1000,ярость,я,ярость я,None


Отметим в `auto_tag` фразы и некириллические слова.  

In [4]:
data.loc[data.word.apply(is_phrase), ["auto_tag"]] = "PHRASE"
data.loc[data.word.apply(is_phrase)]

,id,stim,word,phrase,auto_tag
12,1,абсолютный,без разговора,абсолютный без разговора,PHRASE
32,1,абсолютный,во всем,абсолютный во всем,PHRASE
41,1,абсолютный,высшая точка,абсолютный высшая точка,PHRASE
58,1,абсолютный,жидкость для посуды,абсолютный жидкость для посуды,PHRASE
106,1,абсолютный,не очень,абсолютный не очень,PHRASE
...,...,...,...,...,...
167829,1000,ярость,плохое настроение,ярость плохое настроение,PHRASE
167843,1000,ярость,"сила, глупость","ярость сила, глупость",PHRASE
167844,1000,ярость,сила для победы,ярость сила для победы,PHRASE
167863,1000,ярость,темная сторона силы,ярость темная сторона силы,PHRASE


In [5]:
data.loc[(~data.word.apply(is_word)) & (~data.word.apply(is_phrase)), ["auto_tag"]] = 'NULL'
data.loc[(~data.word.apply(is_word)) & (~data.word.apply(is_phrase))]

,id,stim,word,phrase,auto_tag
0,1,абсолютный,-,абсолютный -,NULL
1,1,абсолютный,0,абсолютный 0,NULL
2,1,абсолютный,100%,абсолютный 100%,NULL
3,1,абсолютный,100%-ный,абсолютный 100%-ный,NULL
4,1,абсолютный,max,абсолютный max,NULL
...,...,...,...,...,...
167436,998,являться,кем-то,являться кем-то,NULL
167439,998,являться,куда-то,являться куда-то,NULL
167575,999,яркий,-,яркий -,NULL
167576,999,яркий,swarowski,яркий swarowski,NULL


Работаем с этим.

In [6]:
data[data.word.apply(is_word)].phrase

5           абсолютный абсолют
6         абсолютный авторитет
7             абсолютный агент
8         абсолютный адреналин
9            абсолютный анализ
                  ...         
167884              ярость шар
167885           ярость эмоции
167886           ярость эмоция
167887                ярость я
167888             ярость ярая
Name: phrase, Length: 140269, dtype: object

In [10]:
data.head(15)

,id,stim,word,phrase,auto_tag
0,1,абсолютный,-,абсолютный -,NULL
1,1,абсолютный,0,абсолютный 0,NULL
2,1,абсолютный,100%,абсолютный 100%,NULL
3,1,абсолютный,100%-ный,абсолютный 100%-ный,NULL
4,1,абсолютный,max,абсолютный max,NULL
5,1,абсолютный,абсолют,абсолютный абсолют,None
6,1,абсолютный,авторитет,абсолютный авторитет,None
7,1,абсолютный,агент,абсолютный агент,None
8,1,абсолютный,адреналин,абсолютный адреналин,None
9,1,абсолютный,анализ,абсолютный анализ,None


Сохраним датасет для будущих поколений (следующих этапов).

In [8]:
data.to_csv('./data/prepared_to_tag.csv', index=False)